# **Module 6:** Generative AI for Cyber Security

## Fine-tunning a model for Cyber Security

Fine-tuning a language model in the context of LLMs (Masked Language Models like BERT), refers to the process of taking a pre-trained language model and further training it on a smaller, task-specific dataset to adapt it to a specific downstream task. The idea is to leverage the knowledge learned during the initial pre-training on a large corpus and then fine-tune the model to perform well on a specific task of interest.

1. **Pre-training on a Large Corpus**

    Initially, the language model is pre-trained on a large and diverse dataset. During this phase, the model learns general language patterns, syntax, and contextual relationships between words.

1. **Task-Specific Data**

    After pre-training, the model is fine-tuned on a smaller dataset that is specific to the task you want the model to perform well on. This dataset is typically labeled and consists of examples relevant to the downstream task.

1. **Architecture and Parameters**

    The architecture of the model remains the same, but the parameters learned during pre-training are further adjusted based on the task-specific data. The fine-tuning process updates the weights of the model to make it more suited for the specific task.


1. **Task-Specific Objective Function**

    The objective function used during fine-tuning is tailored to the downstream task. For example, in classification tasks, the model might be fine-tuned using a cross-entropy loss function.

1. **Learning Rate and Training Hyperparameters**

    Fine-tuning often involves adjusting the learning rate and other hyperparameters to ensure effective training on the smaller dataset. This helps prevent overfitting and encourages the model to adapt to the specific task.
   
1. **Transfer of Knowledge**

    The knowledge gained during pre-training, such as understanding of language structures and semantics, is transferred to the task-specific model. Fine-tuning allows the model to specialize without losing the general language understanding acquired during pre-training.

![](https://www.labellerr.com/blog/content/images/2023/08/Fine-tune-example.png)

Fine-tuning is particularly useful when you have a limited amount of task-specific data. By starting with a pre-trained model, you can benefit from the knowledge embedded in the model and fine-tune it to achieve good performance on your specific task, even with a smaller dataset. This approach has been successful in various natural language processing (NLP) tasks, ranging from sentiment analysis to named entity recognition.

## Case Study: SecureBret

[SecureBERT](https://arxiv.org/pdf/2204.02685) is a domain-specific language model to represent cybersecurity textual data which is trained on a large amount of in-domain text crawled from online resources.

SecureBERT can be used as the base model for any downstream task including text classification, NER, Seq-to-Seq, QA, etc.
- SecureBERT has demonstrated significantly higher performance in predicting masked words within the text when compared to existing models like RoBERTa (base and large), SciBERT, and SecBERT.
- SecureBERT has also demonstrated promising performance in preserving general English language understanding (representation).

![](https://user-images.githubusercontent.com/46252665/195998237-9bbed621-8002-4287-ac0d-19c4f603d919.png)

In [101]:
!pip install transformers
!pip install torch
!pip install tokenizers

In [102]:
from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
model = RobertaModel.from_pretrained("ehsanaghaei/SecureBERT")

inputs = tokenizer("This is SecureBERT!", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state


import torch
import transformers
from transformers import RobertaTokenizer, RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("ehsanaghaei/SecureBERT")
model = transformers.RobertaForCausalLM.from_pretrained("ehsanaghaei/SecureBERT")


def predict_mask(sent, tokenizer, model, topk=10, print_results=True):
    token_ids = tokenizer.encode(sent, return_tensors="pt")
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position]
    words = []
    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list = []
    for index, mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=topk, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        words = [w.replace(" ", "") for w in words]
        list_of_list.append(words)

    return list_of_list


from IPython.display import display, HTML
import html


def input_masked_sentence(input):
    def escape_mask(text):
        return text.replace("<mask>", "<&zwj;mask>")

    display(HTML(f"<b>Input:</b> {escape_mask(input)}"))
    for output in predict_mask(input, tokenizer, model):
        display(HTML(f"<b>SecureBert:</b> {' | '.join(output)}"))

Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
If you want 

The `<mask>` token is commonly used in the context of masked language models (LLMs) or masked language model pre-training. This approach is a type of unsupervised learning where a model is trained to predict missing or masked tokens in a sequence of text. The `<mask>` token is used to represent the positions in the input text where tokens are masked or hidden during training.

Here's a general overview of how the `<mask>` token works in the context of LLMs:

**Masking during Training:**

During the pre-training phase, a certain percentage of tokens in the input text are randomly selected to be masked. These masked tokens are then replaced with the `<mask>` token.
The model is trained to predict the original identity of the masked tokens based on the context provided by the surrounding tokens.
Objective Function:

The objective function during training is typically to maximize the likelihood of predicting the correct tokens at the masked positions.
This training process helps the model learn contextual relationships and dependencies between words in a given language.

![](https://user-images.githubusercontent.com/46252665/195998153-f5682f7c-60a8-486d-b2c1-9ef5732c24ba.png)

**Fine-Tuning and Downstream Tasks:**

After pre-training, the model can be fine-tuned on specific downstream tasks (such as text classification, named entity recognition, etc.).
The knowledge gained during pre-training helps the model perform well on a range of natural language processing (NLP) tasks.
Prediction during Inference:

During inference or when using the model for downstream tasks, the `<mask>` token can be used to predict missing tokens in a given sequence. For example, if you provide a sentence with some tokens replaced by `<mask>`, the model can predict the most likely tokens for those masked positions.
Overall, the `<mask>` token is a key element in the training process of LLMs, enabling them to learn rich representations of language and perform well on a variety of NLP tasks. The most well-known model that uses this approach is BERT (Bidirectional Encoder Representations from Transformers).



In [103]:
input_masked_sentence(
    "Adversaries may also compromise sites then include <mask> content designed to collect website authentication cookies from visitors."
)

In [104]:
input_masked_sentence(
    "One example of this is MS14-068, which targets <mask> and can be used to forge Kerberos tickets using domain user permissions."
)

In [105]:
input_masked_sentence("Paris is the <mask> of France.")

In [106]:
input_masked_sentence("Virus causes <mask>.")

In [107]:
input_masked_sentence("Sending huge amount of packets through network leads to <mask>.")

In [108]:
input_masked_sentence(
    "A <mask> injection occurs when an attacker inserts malicious code into a server"
)

## Using LLMs as Offensive Generative AI

## Offensive Generative AI Model

Offensive security is the practice of actively seeking out vulnerabilities in an organization's cybersecurity. It often involves using similar tactics as attackers and might include red teaming, penetration testing and vulnerability assessments. Offensive security can be shortened to "OffSec."

Offensive generative AI refers natural language processing (NLP) and machine learning techniques, that are designed or trained to produce content that is offensive, harmful, or inappropriate in some way. This content could include hate speech, derogatory language, violent or threatening messages, misinformation, or other forms of harmful content. Malicious AI can weaponising generative AI to improve the monetisation of their attacks, which will stem from a surge in ransomware attacks and phishing campaigns.

![image.png](https://canalys-prod-public.s3.eu-west-1.amazonaws.com/client/static/uimages/7Xb9lz2jbGfvTH1TJcfoGuID7VNqyocPICtqhesEJqerayYVsmLORCMJb1BQNSKC.png)

Such AI systems can pose significant ethical and societal challenges. They have the potential to spread harmful messages at scale, amplify existing biases and prejudices present in the training data, and contribute to the proliferation of toxic online environments. Offensive generative AI can be used maliciously by individuals or groups to harass others, manipulate public opinion, or undermine trust in information sources.


### Generative model for cracking password

Generative AI poses several dangers for password cracking:

- Speed and Efficiency: Generative AI can significantly speed up the process of password cracking by generating and testing a vast number of possible passwords in a short amount of time. This can make it much more efficient for attackers to breach password-protected systems or accounts.

- Sophisticated Attack Techniques: Generative AI can employ sophisticated techniques, such as neural networks or reinforcement learning, to generate passwords that are more likely to be successful in cracking password hashes or bypassing authentication mechanisms.

- Adaptability: Generative AI models can adapt to different patterns and structures commonly found in passwords, such as common words, phrases, or character combinations. This adaptability makes them more effective at cracking passwords that may not be easily guessable or vulnerable to traditional brute-force methods.

- Privacy Risks: If generative AI is used for password cracking, it could compromise the privacy and security of individuals or organizations by gaining unauthorized access to sensitive information stored behind password-protected systems or accounts.

- Scale and Automation: Generative AI can be deployed at scale and automated, allowing attackers to launch large-scale password cracking attacks against multiple targets simultaneously, increasing the potential impact and severity of security breaches.imators.

### PassGPT


PassGPT is an LLM trained on password leaks for password generation. PassGPT outperforms existing methods based **by guessing twice as many previously unseen passwords**. PassGPT also contains the concept of guided password generation, where they leverage PassGPT sampling procedure to generate passwords matching arbitrary constraints.


![](https://raw.githubusercontent.com/WSU-AI-CyberSecurity/WSU-AI-CyberSecurity.github.io/main/assets/passgpt.png)


In [109]:
from transformers import GPT2LMHeadModel
from transformers import RobertaTokenizerFast
import torch

tokenizer = RobertaTokenizerFast.from_pretrained(
    "javirandor/passgpt-10characters",
    max_len=12,
    padding="max_length",
    truncation=True,
    do_lower_case=False,
    strip_accents=False,
    mask_token="<mask>",
    unk_token="<unk>",
    pad_token="<pad>",
    truncation_side="right",
)

model = GPT2LMHeadModel.from_pretrained("javirandor/passgpt-10characters").eval()

NUM_GENERATIONS = 1


def generate_password(prefix, max_length=12):
    start_token = tokenizer.encode(prefix)
    start_token = start_token[0:-1]

    # Generate passwords sampling from the beginning of password token
    g = model.generate(
        torch.tensor([start_token]),
        # do_sample=True,
        num_return_sequences=NUM_GENERATIONS,
        max_length=max_length + 1,
        pad_token_id=tokenizer.pad_token_id,
        bad_words_ids=[[tokenizer.bos_token_id]],
    )

    # Remove start of sentence token
    g = g[:, 1:]

    decoded = tokenizer.batch_decode(g.tolist())
    decoded_clean = [
        i.split("</s>")[0] for i in decoded
    ]  # Get content before end of password token

    # Print your sampled passwords!
    return decoded_clean


seedgen = torch.manual_seed(0)

### Learning the behaviour of human's password creation

The following are generating the most likely password that people choose based on big data

In [110]:
generate_password("")

['0876060007']

In [111]:
generate_password(prefix="pas", max_length=8)

['password']

In [112]:
generate_password(prefix="qw", max_length=6)

['qwerty']

In [113]:
generate_password(prefix="ilo", max_length=12)

['iloveme123']

In [114]:
generate_password(prefix="John", max_length=12)

['Johnny1994']

**You can also do a conditional password generation**

In [115]:
import string

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE = "cpu"

# Map each of the desired character groups into their corresponding ids (as given by the tokenizer)
lowercase = list(string.ascii_lowercase)
uppercase = list(string.ascii_uppercase)
digits = list(string.digits)
punctuation = list(string.punctuation)

lowercase_tokens = tokenizer(lowercase, add_special_tokens=False).input_ids
uppercase_tokens = tokenizer(uppercase, add_special_tokens=False).input_ids
digits_tokens = tokenizer(digits, add_special_tokens=False).input_ids
punctuation_tokens = tokenizer(punctuation, add_special_tokens=False).input_ids


# All possible tokens in our model
all_tokens = [[i] for i in range(len(tokenizer))]


def conditional_generation(template, num_generations=1):
    generated = 0
    generations = []

    while generated < num_generations:
        generation = torch.tensor([tokenizer.bos_token_id]).unsqueeze(0)
        current_length = 1

        for char in template:
            if char == "l":
                bad_tokens = [i for i in all_tokens if i not in lowercase_tokens]
            elif char == "u":
                bad_tokens = [i for i in all_tokens if i not in uppercase_tokens]
            elif char == "d":
                bad_tokens = [i for i in all_tokens if i not in digits_tokens]
            elif char == "p":
                bad_tokens = [i for i in all_tokens if i not in punctuation_tokens]
            else:
                bad_tokens = [[tokenizer.eos_token_id]]

            generation = model.generate(
                generation.to(DEVICE),
                do_sample=True,
                max_length=current_length + 1,
                pad_token_id=tokenizer.pad_token_id,
                num_return_sequences=1,
                bad_words_ids=bad_tokens,
            )
            current_length += 1

        if not 2 in generation.flatten():
            generations.append(generation)
            generated += 1

    return tokenizer.batch_decode(torch.cat(generations, 0)[:, 1:])

### Conditional Password Generation
One of the main advantages of Generative AI is the possibility of generating passwords under arbitrary constraints. In this template code, we have created five different groups of characters that we can sample from at each position:

- `l`: lowercase letters
- `u`: uppercase letters
- `d`: digits
- `p`: punctuation
- `*`: any character in the vocabulary
- 
You can create any template by combining these. For example, `lllldd` will generate passwords starting with four lowercase letters and finishing with two digits.

Feel free to create your own character groups below.

In [116]:
conditional_generation("lllldd", 5)

['owen32', 'will14', 'bubb12', 'rgdl15', 'pinc04']

In [118]:
conditional_generation("u*****lldd", 5)

['PSYCHOfr12', 'Lanenabb12', 'Pochacho99', 'SHADOWlu69', 'JayDanki07']